# AdventureWorks - Medium

In [1]:
import getpass
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
pwd = getpass.getpass()
engine = create_engine(
    'postgresql+psycopg2://postgres:%s@192.168.31.31:15432/sqlzoo' % (pwd))
pd.set_option('display.max_rows', 60)

········


In [2]:
cust_aw = pd.read_sql_table('CustomerAW', engine)
cust_addr = pd.read_sql_table('CustomerAddress', engine)
addr = pd.read_sql_table('Address', engine)
product = pd.read_sql_table('Product', engine)
order_det = pd.read_sql_table('SalesOrderDetail', engine)
order_head = pd.read_sql_table('SalesOrderHeader', engine)
prod_model = pd.read_sql_table('ProductModel', engine)
prod_model_prod = pd.read_sql_table('ProductModelProductDescription', engine)
prod_desc = pd.read_sql_table('ProductDescription', engine)
prod_cat = pd.read_sql_table('ProductCategory', engine)

## 6.
A "Single Item Order" is a customer order where only one item is ordered. Show the SalesOrderID and the UnitPrice for every Single Item Order.

In [3]:
a = (order_det.loc[order_det['OrderQty']==1]
     .groupby(['SalesOrderID', 'UnitPrice'])
     .agg(n=('SalesOrderDetailID', 'count'))
     .reset_index())

a.loc[a['n']==1, ['SalesOrderID', 'UnitPrice']]

,SalesOrderID,UnitPrice
1,71776,63.90
2,71780,32.99
3,71780,48.59
4,71780,72.16
5,71780,113.00
...,...,...
97,71935,323.99
98,71936,20.99
99,71936,113.00
100,71936,149.87


## 7.
Where did the racing socks go? List the product name and the CompanyName for all Customers who ordered ProductModel 'Racing Socks'.

In [4]:
a = (product.merge(order_det, on='ProductID')
     .merge(order_head, on='SalesOrderID')
     .merge(cust_aw, on='CustomerID')
     .merge(prod_model, on='ProductModelID'))

(a.loc[a['Name_y']=='Racing Socks', ['CompanyName', 'Name_x']]
 .sort_values('CompanyName').drop_duplicates())

,CompanyName,Name_x
281,Eastside Department Store,"Racing Socks, L"
290,Essential Bike Works,"Racing Socks, L"
288,Remarkable Bike Store,"Racing Socks, M"
289,Remarkable Bike Store,"Racing Socks, L"
282,Riding Cycles,"Racing Socks, L"
283,Sports Products Store,"Racing Socks, M"
284,Sports Products Store,"Racing Socks, L"
286,The Bicycle Accessories Company,"Racing Socks, M"
287,The Bicycle Accessories Company,"Racing Socks, L"
285,Thrifty Parts and Sales,"Racing Socks, M"


## 8.
Show the product description for culture 'fr' for product with ProductID 736.

In [5]:
a = (product.merge(prod_model_prod, on='ProductModelID')
     .merge(prod_desc, on='ProductDescriptionID'))

a.loc[(a['ProductID']==736) & (a['Culture'].str.contains('fr')), ['Description']]

,Description
159,Le cadre LL en aluminium offre une conduite co...


## 9.
Use the SubTotal value in SaleOrderHeader to list orders from the largest to the smallest. For each order show the CompanyName and the SubTotal and the total weight of the order.

In [6]:
a = (order_head.merge(order_det, on='SalesOrderID')
     .merge(product, on='ProductID')
     .merge(cust_aw, on='CustomerID'))
a['Weight'] = a['Weight'].fillna(0) * a['OrderQty'].fillna(0)
(a.groupby(['CompanyName', 'SubTotal'])['Weight'].sum()
 .reset_index().sort_values('SubTotal', ascending=False))

,CompanyName,SubTotal,Weight
0,Action Bicycle Specialists,108561.83,1133911.56
15,Metropolitan Bicycle Supply,98278.69,679588.02
2,Bulk Discount Store,88812.86,34813.05
7,Eastside Department Store,83858.43,565638.72
20,Riding Cycles,78029.69,504095.33
14,Many Bikes Store,74058.81,744328.60
13,Instruments and Parts Company,63980.99,731576.77
10,Extreme Riding Supplies,57634.63,589939.11
27,Trailblazing Sports,41622.05,234328.12
18,Professional Sales and Service,39785.33,396843.63


## 10.
How many products in ProductCategory 'Cranksets' have been sold to an address in 'London'?

In [7]:
a = (product.merge(prod_cat.loc[prod_cat['name']=='Cranksets'], 
                   on='ProductCategoryID')
     .merge(order_det, on='ProductID')
     .merge(order_head, on='SalesOrderID')
     .merge(cust_aw, on='CustomerID')
     .merge(cust_addr, on='CustomerID')
     .merge(addr.loc[addr['City']=='London'], on='AddressID'))

pd.DataFrame({'count': [a.shape[0]]})


,count
0,2
